## Procesamiento Refined
### Objetivos
- Limpieza de datos de archivos en zona raw
- Enriquecimiento de datos

In [2]:
# Importamos Pandas y cargamos archivos
import pandas as pd
import numpy as np

# Cargamos el archivo de medallas
df_medals = pd.read_csv('datos/raw/olympic_medals.csv')

# Cargamos el archivo de resultados
df_results = pd.read_csv('datos/raw/olympic_results_from_pkl.csv')

# Cargamos el archivo de atletas
df_atletas = pd.read_csv('datos/raw/olympic_athletes.csv')

# Cargamos el archivo de hosts
df_hosts = pd.read_csv('datos/raw/olympic_hosts.csv')

Definimos Funciones a Utilizar en varios DataFrames

In [3]:
def spacer():
    """
    Imprime en pantalla un separador horizontal
    """
    # print("\n")
    print("===============================================")
    # print("\n")


def generar_df_resumen(df):
    """
    Genera un resumen de las columnas de un DataFrame.

    Este método crea un resumen de las columnas de un DataFrame, incluyendo
    nombre de la columna, tipo de dato, cantidad de valores, cantidad de valores únicos,
    cantidad de valores nulos, cantidad de strings vacíos, valor más frecuente y valor menos frecuente.

    :param df: El DataFrame a resumir.
    :type df: pd.DataFrame
    :return: Un DataFrame con el resumen de las columnas.
    :rtype: pd.DataFrame
    """
    # Crear una lista para almacenar el resumen de las columnas
    resumen_data = []

    for col in df.columns:
        nombre = col
        tipo = df[col].dtype
        cantidad_valores = len(df[col])
        valores_unicos = df[col].apply(lambda x: str(x) if isinstance(x, list) else x).nunique()
        nulos = df[col].isnull().sum()
        strings_vacios = df[col].apply(lambda x: x == '').sum() if df[col].dtype == 'object' else 0

        # Tratar de obtener el valor más frecuente
        try:
            mas_frecuente = df[col].mode().iloc[0] if not df[col].mode().empty else None
        except TypeError:
            mas_frecuente = None

        # Tratar de obtener el valor menos frecuente
        try:
            menos_frecuente = df[col].value_counts().idxmin() if not df[col].value_counts().empty else None
        except TypeError:
            menos_frecuente = None

        # Añadir la información de la columna al resumen
        resumen_data.append({
            'Nombre': nombre,
            'Tipo': tipo,
            'Cantidad de Valores': cantidad_valores,
            'Valores Únicos': valores_unicos,
            'Nulos': nulos,
            'Strings Vacíos': strings_vacios,
            'Más Frecuente': mas_frecuente,
            'Menos Frecuente': menos_frecuente
        })

    # Convertir la lista de resumen a un DataFrame
    resumen = pd.DataFrame(resumen_data)

    # Devolver el resumen
    return resumen


def analizar_dataframe(df, nombre):
    """
    Realiza un análisis inicial de un DataFrame.

    Este método imprime y muestra en pantalla las primeras filas del DataFrame,
    información básica del DataFrame, descripción estadística y un resumen detallado
    de las columnas del DataFrame.

    :param df: El DataFrame a analizar.
    :type df: pd.DataFrame
    :param nombre: El nombre del DataFrame para usar en los títulos de la salida.
    :type nombre: str
    """
    print(f"====== Dataset: {nombre} ======")
    print("Primeras 10 filas:")
    display(df.head(10))
    spacer()
    print(f"\nInformación Básica del DataFrame: {nombre}")
    df.info()
    spacer()
    print("\nDescripción Estadística:")
    display(df.describe(include='all'))
    spacer()
    print("\nDatos de Columnas:")
    display(generar_df_resumen(df))


def verificar_limpieza(df):
    """
    Imprime en pantalla información sobre nulos y tipos de datos en cada columna de un DataFrame.

    :param df: El DataFrame a verificar.
    :type df: pd.DataFrame
    """
    pd.set_option('display.max_columns', None)
    print("Datos Nulos:")
    print(df.isnull().sum())
    print("\n")
    print("Tipos de Datos:")
    print(df.dtypes)


def trim_strings(df):
    """
    Elimina espacios en blanco antes y después de todas las columnas de tipo string en un DataFrame.

    :param df: El DataFrame en el cual se eliminarán los espacios en blanco.
    :type df: pd.DataFrame
    :return: Un nuevo DataFrame con los espacios en blanco eliminados en las columnas de tipo string.
    :rtype: pd.DataFrame
    """
    # Seleccionar solo las columnas de tipo string
    string_columns = df.select_dtypes(include=['object']).columns
    
    # Aplicar la función strip solo a los valores que son realmente strings
    for col in string_columns:
        df[col] = df[col].apply(lambda x: x.strip() if isinstance(x, str) else x)
    
    return df


def sustituir_valor_filtrado(df, value, filtro):
    """
    Sustituye los valores nulos o vacíos en una columna específica con valores existentes de la misma columna,
    utilizando como referencia otra columna relacionada.

    :param df: El DataFrame que contiene los datos.
    :type df: pd.DataFrame
    :param value: El nombre de la columna donde se desea sustituir los valores nulos o vacíos.
    :type value: str
    :param filtro: El nombre de la columna utilizada como referencia para encontrar los valores de sustitución.
    :type filtro: str
    :return: El DataFrame con los valores nulos o vacíos sustituidos en la columna especificada.
    :rtype: pd.DataFrame
    """
    # Identificar las filas que tienen valor nulo o vacío en la columna 'value'
    mask = pd.isnull(df[value]) | (df[value] == "")
    # Filtrar el DataFrame original para obtener solo las filas donde 'value' es nulo o vacío
    df_nulls = df[mask]
    # Obtener los valores únicos de 'filter' para iterar sobre ellos
    unique_filters = df_nulls[filtro].unique()
    # Iterar sobre cada valor único de 'filter' y rellenar los valores nulos o vacíos correspondientes
    for f in unique_filters:
        # Filtrar las filas donde 'filter' coincide y 'value' no es nulo ni vacío
        mask_filter = (df[filtro] == f) & pd.notna(df[value]) & (df[value] != "")
        df_filter = df[mask_filter]

        # Si hay filas en df_filter, asignar el valor de la primera fila a las filas correspondientes en df_nulls
        if not df_filter.empty:
            first_value = df_filter.iloc[0][value]
            df.loc[mask & (df[filtro] == f), value] = first_value

    return df


def llenar_url_faltante(df_rellenar, df_referencia_atletas):
    """
    Rellena los valores nulos o vacíos en la columna 'athlete_url' de un DataFrame utilizando las URLs disponibles
    en otro DataFrame basado en el nombre completo del atleta.

    :param df_rellenar: El DataFrame que contiene los datos a completar.
    :type df_rellenar: pd.DataFrame
    :param df_referencia_atletas: El DataFrame que contiene las URLs de los atletas.
    :type df_referencia_atletas: pd.DataFrame
    :return: El DataFrame con los valores nulos en 'athlete_url' completados.
    :rtype: pd.DataFrame
    """
    # Crear un diccionario de nombres de atletas a URLs para evitar múltiples accesos a df_referencia_atletas
    nombre_a_url = df_referencia_atletas.set_index('athlete_full_name')['athlete_url'].to_dict()

    # Función para obtener la URL o cadena vacía si no se encuentra
    def obtener_url(nombre):
        return nombre_a_url.get(nombre, '')

    # Máscara de filas donde athlete_url es nulo o vacío en df_rellenar
    mask = df_rellenar['athlete_url'].isnull() | (df_rellenar['athlete_url'] == "")

    # Llenar los valores nulos en df_rellenar con las URLs encontradas en df_referencia_atletas
    df_rellenar.loc[mask, 'athlete_url'] = df_rellenar.loc[mask, 'athlete_full_name'].apply(obtener_url)

    return df_rellenar



Limpieza Dataset Hosts

In [4]:
analizar_dataframe(df_hosts, "Hosts")

====== Dataset: Hosts ======
Primeras 10 filas:


,game_slug,game_end_date,game_start_date,game_location,game_name,game_season,game_year
0,beijing-2022,2022-02-20T12:00:00Z,2022-02-04T15:00:00Z,China,Beijing 2022,Winter,2022
1,tokyo-2020,2021-08-08T14:00:00Z,2021-07-23T11:00:00Z,Japan,Tokyo 2020,Summer,2020
2,pyeongchang-2018,2018-02-25T08:00:00Z,2018-02-08T23:00:00Z,Republic of Korea,PyeongChang 2018,Winter,2018
3,rio-2016,2016-08-21T21:00:00Z,2016-08-05T12:00:00Z,Brazil,Rio 2016,Summer,2016
4,sochi-2014,2014-02-23T16:00:00Z,2014-02-07T04:00:00Z,Russian Federation,Sochi 2014,Winter,2014
5,london-2012,2012-08-12T19:00:00Z,2012-07-27T07:00:00Z,Great Britain,London 2012,Summer,2012
6,vancouver-2010,2010-02-28T04:00:00Z,2010-02-12T16:00:00Z,Canada,Vancouver 2010,Winter,2010
7,beijing-2008,2008-08-24T12:00:00Z,2008-08-08T00:00:00Z,China,Beijing 2008,Summer,2008
8,turin-2006,2006-02-26T19:00:00Z,2006-02-10T07:00:00Z,Italy,Turin 2006,Winter,2006
9,athens-2004,2004-08-29T18:00:00Z,2004-08-13T06:00:00Z,Greece,Athens 2004,Summer,2004



Información Básica del DataFrame: Hosts
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53 entries, 0 to 52
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   game_slug        53 non-null     object
 1   game_end_date    53 non-null     object
 2   game_start_date  53 non-null     object
 3   game_location    53 non-null     object
 4   game_name        53 non-null     object
 5   game_season      53 non-null     object
 6   game_year        53 non-null     int64 
dtypes: int64(1), object(6)
memory usage: 3.0+ KB

Descripción Estadística:


,game_slug,game_end_date,game_start_date,game_location,game_name,game_season,game_year
count,53,53,53,53,53,53,53.000000
unique,53,53,53,26,53,2,NaN
top,beijing-2022,2022-02-20T12:00:00Z,2022-02-04T15:00:00Z,United States,Beijing 2022,Summer,NaN
freq,1,1,1,8,1,29,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,1967.547170
std,NaN,NaN,NaN,NaN,NaN,NaN,35.201926
min,NaN,NaN,NaN,NaN,NaN,NaN,1896.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,1936.000000
50%,NaN,NaN,NaN,NaN,NaN,NaN,1972.000000
75%,NaN,NaN,NaN,NaN,NaN,NaN,1996.000000



Datos de Columnas:


,Nombre,Tipo,Cantidad de Valores,Valores Únicos,Nulos,Strings Vacíos,Más Frecuente,Menos Frecuente
0,game_slug,object,53,53,0,0,albertville-1992,beijing-2022
1,game_end_date,object,53,53,0,0,1896-04-15T11:39:39Z,2022-02-20T12:00:00Z
2,game_start_date,object,53,53,0,0,1896-04-06T11:38:39Z,2022-02-04T15:00:00Z
3,game_location,object,53,26,0,0,United States,Russian Federation
4,game_name,object,53,53,0,0,Albertville 1992,Beijing 2022
5,game_season,object,53,2,0,0,Summer,Winter
6,game_year,int64,53,37,0,0,1924,2014


In [5]:
# Convertimos las columnas a un tipo de dato más específico 
# obviamos las conversiones a str, usamos tipo object
df_hosts['game_end_date'] = pd.to_datetime(df_hosts['game_end_date'], format='%Y-%m-%dT%H:%M:%SZ')
df_hosts['game_start_date'] = pd.to_datetime(df_hosts['game_start_date'], format='%Y-%m-%dT%H:%M:%SZ')
df_hosts['game_year'] = df_hosts['game_year'].astype(int)

In [6]:
# Limpiamos posibles espacios en blanco al principio y final de strings
df_hosts = trim_strings(df_hosts)

In [7]:
# Verificamos resultados
analizar_dataframe(df_hosts, "Hosts")
verificar_limpieza(df_hosts)

====== Dataset: Hosts ======
Primeras 10 filas:


,game_slug,game_end_date,game_start_date,game_location,game_name,game_season,game_year
0,beijing-2022,2022-02-20 12:00:00,2022-02-04 15:00:00,China,Beijing 2022,Winter,2022
1,tokyo-2020,2021-08-08 14:00:00,2021-07-23 11:00:00,Japan,Tokyo 2020,Summer,2020
2,pyeongchang-2018,2018-02-25 08:00:00,2018-02-08 23:00:00,Republic of Korea,PyeongChang 2018,Winter,2018
3,rio-2016,2016-08-21 21:00:00,2016-08-05 12:00:00,Brazil,Rio 2016,Summer,2016
4,sochi-2014,2014-02-23 16:00:00,2014-02-07 04:00:00,Russian Federation,Sochi 2014,Winter,2014
5,london-2012,2012-08-12 19:00:00,2012-07-27 07:00:00,Great Britain,London 2012,Summer,2012
6,vancouver-2010,2010-02-28 04:00:00,2010-02-12 16:00:00,Canada,Vancouver 2010,Winter,2010
7,beijing-2008,2008-08-24 12:00:00,2008-08-08 00:00:00,China,Beijing 2008,Summer,2008
8,turin-2006,2006-02-26 19:00:00,2006-02-10 07:00:00,Italy,Turin 2006,Winter,2006
9,athens-2004,2004-08-29 18:00:00,2004-08-13 06:00:00,Greece,Athens 2004,Summer,2004



Información Básica del DataFrame: Hosts
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53 entries, 0 to 52
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   game_slug        53 non-null     object        
 1   game_end_date    53 non-null     datetime64[ns]
 2   game_start_date  53 non-null     datetime64[ns]
 3   game_location    53 non-null     object        
 4   game_name        53 non-null     object        
 5   game_season      53 non-null     object        
 6   game_year        53 non-null     int64         
dtypes: datetime64[ns](2), int64(1), object(4)
memory usage: 3.0+ KB

Descripción Estadística:


,game_slug,game_end_date,game_start_date,game_location,game_name,game_season,game_year
count,53,53,53,53,53,53,53.000000
unique,53,NaN,NaN,26,53,2,NaN
top,beijing-2022,NaN,NaN,United States,Beijing 2022,Summer,NaN
freq,1,NaN,NaN,8,1,29,NaN
mean,NaN,1967-12-29 22:12:03.735849056,1967-11-30 15:08:28.641509432,NaN,NaN,NaN,1967.547170
min,NaN,1896-04-15 11:39:39,1896-04-06 11:38:39,NaN,NaN,NaN,1896.000000
25%,NaN,1936-08-16 19:00:00,1936-08-01 07:00:00,NaN,NaN,NaN,1936.000000
50%,NaN,1972-02-13 11:00:00,1972-02-02 23:00:00,NaN,NaN,NaN,1972.000000
75%,NaN,1996-08-05 21:00:00,1996-07-19 12:00:00,NaN,NaN,NaN,1996.000000
max,NaN,2022-02-20 12:00:00,2022-02-04 15:00:00,NaN,NaN,NaN,2022.000000



Datos de Columnas:


,Nombre,Tipo,Cantidad de Valores,Valores Únicos,Nulos,Strings Vacíos,Más Frecuente,Menos Frecuente
0,game_slug,object,53,53,0,0,albertville-1992,beijing-2022
1,game_end_date,datetime64[ns],53,53,0,0,1896-04-15 11:39:39,2022-02-20 12:00:00
2,game_start_date,datetime64[ns],53,53,0,0,1896-04-06 11:38:39,2022-02-04 15:00:00
3,game_location,object,53,26,0,0,United States,Russian Federation
4,game_name,object,53,53,0,0,Albertville 1992,Beijing 2022
5,game_season,object,53,2,0,0,Summer,Winter
6,game_year,int64,53,37,0,0,1924,2014


Datos Nulos:
game_slug          0
game_end_date      0
game_start_date    0
game_location      0
game_name          0
game_season        0
game_year          0
dtype: int64


Tipos de Datos:
game_slug                  object
game_end_date      datetime64[ns]
game_start_date    datetime64[ns]
game_location              object
game_name                  object
game_season                object
game_year                   int64
dtype: object


Limpieza Dataset Resultados

In [8]:
analizar_dataframe(df_results, "Resultados")

====== Dataset: Resultados ======
Primeras 10 filas:


,discipline_title,event_title,slug_game,participant_type,medal_type,athletes,rank_equal,rank_position,country_name,country_code,country_3_letter_code,athlete_url,athlete_full_name,value_unit,value_type
0,Curling,Mixed Doubles,beijing-2022,GameTeam,GOLD,"[('Stefania CONSTANTINI', 'https://olympics.co...",False,1,Italy,IT,ITA,NaN,NaN,NaN,NaN
1,Curling,Mixed Doubles,beijing-2022,GameTeam,SILVER,"[('Kristin SKASLIEN', 'https://olympics.com/en...",False,2,Norway,NO,NOR,NaN,NaN,NaN,NaN
2,Curling,Mixed Doubles,beijing-2022,GameTeam,BRONZE,"[('Almida DE VAL', 'https://olympics.com/en/at...",False,3,Sweden,SE,SWE,NaN,NaN,NaN,NaN
3,Curling,Mixed Doubles,beijing-2022,GameTeam,NaN,"[('Jennifer DODDS', 'https://olympics.com/en/a...",False,4,Great Britain,GB,GBR,NaN,NaN,NaN,NaN
4,Curling,Mixed Doubles,beijing-2022,GameTeam,NaN,"[('Rachel HOMAN', 'https://olympics.com/en/ath...",False,5,Canada,CA,CAN,NaN,NaN,NaN,NaN
5,Curling,Mixed Doubles,beijing-2022,GameTeam,NaN,"[('Zuzana HAJKOVA', 'https://olympics.com/en/a...",False,6,Czech Republic,CZ,CZE,NaN,NaN,NaN,NaN
6,Curling,Mixed Doubles,beijing-2022,GameTeam,NaN,"[('Jenny PERRET', 'https://olympics.com/en/ath...",False,7,Switzerland,CH,SUI,NaN,NaN,NaN,NaN
7,Curling,Mixed Doubles,beijing-2022,GameTeam,NaN,"[('Vicky PERSINGER', 'https://olympics.com/en/...",False,8,United States of America,US,USA,NaN,NaN,NaN,NaN
8,Curling,Mixed Doubles,beijing-2022,GameTeam,NaN,"[('Suyuan FAN', 'https://olympics.com/en/athle...",False,9,People's Republic of China,CN,CHN,NaN,NaN,NaN,NaN
9,Curling,Mixed Doubles,beijing-2022,GameTeam,NaN,"[('Tahli GILL', 'https://olympics.com/en/athle...",False,10,Australia,AU,AUS,NaN,NaN,NaN,NaN



Información Básica del DataFrame: Resultados
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162804 entries, 0 to 162803
Data columns (total 15 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   discipline_title       162804 non-null  object
 1   event_title            162804 non-null  object
 2   slug_game              162804 non-null  object
 3   participant_type       162804 non-null  object
 4   medal_type             20206 non-null   object
 5   athletes               7976 non-null    object
 6   rank_equal             32526 non-null   object
 7   rank_position          158926 non-null  object
 8   country_name           162804 non-null  object
 9   country_code           157768 non-null  object
 10  country_3_letter_code  162804 non-null  object
 11  athlete_url            129991 non-null  object
 12  athlete_full_name      141646 non-null  object
 13  value_unit             78646 non-null   object
 14  value_

,discipline_title,event_title,slug_game,participant_type,medal_type,athletes,rank_equal,rank_position,country_name,country_code,country_3_letter_code,athlete_url,athlete_full_name,value_unit,value_type
count,162804,162804,162804,162804,20206,7976,32526,158926,162804,157768,162804,129991,141646,78646,90049
unique,86,1567,53,2,3,6845,2,193,238,213,233,68104,75263,46475,12
top,Athletics,individual mixed,rio-2016,Athlete,BRONZE,"[('LETOT LETOT', 'https://olympics.com/en/athl...",True,9,United States of America,US,USA,https://olympics.com/en/athletes/heikki-savola...,Heikki SAVOLAINEN,same time,TIME
freq,19259,2731,7672,141646,7029,37,23613,8279,11916,11916,11916,34,34,99,43774



Datos de Columnas:


,Nombre,Tipo,Cantidad de Valores,Valores Únicos,Nulos,Strings Vacíos,Más Frecuente,Menos Frecuente
0,discipline_title,object,162804,86,0,0,Athletics,Basque Pelota
1,event_title,object,162804,1567,0,0,individual mixed,10m rating 1919 mixed
2,slug_game,object,162804,53,0,0,rio-2016,chamonix-1924
3,participant_type,object,162804,2,0,0,Athlete,GameTeam
4,medal_type,object,162804,3,142598,0,BRONZE,SILVER
5,athletes,object,162804,6845,154828,0,"[('LETOT LETOT', 'https://olympics.com/en/athl...","[('Stefania CONSTANTINI', 'https://olympics.co..."
6,rank_equal,object,162804,2,130278,0,True,False
7,rank_position,object,162804,193,3878,0,9,YCD
8,country_name,object,162804,238,0,0,United States of America,Eswatini
9,country_code,object,162804,213,5036,0,US,SS


In [9]:
# Limpiamos posibles espacios en blanco al principio y final de strings
df_results = trim_strings(df_results)

# Reemplazar valores nulos en 'rank_equal' con False usando np.where
# Esto evita el FutureWarning relacionado con el downcasting en fillna
df_results['rank_equal'] = np.where(df_results['rank_equal'].isnull(), False, df_results['rank_equal'])
# Convertir la columna 'rank_equal' a booleano explícitamente
df_results['rank_equal'] = df_results['rank_equal'].astype(bool)

# Reemplazar valores nulos en 'rank_position' con 'NM'
df_results['rank_position'] = df_results['rank_position'].fillna('NM')

In [10]:
# Remplazamos valores vacíos en la columna country_code por valores country_code cuando country_3_letter_code es igual en ambas filas
df_results = sustituir_valor_filtrado(df_results, 'country_code', 'country_3_letter_code')

# Remplazamos valores vacíos restantes en la columna country_code por valores country_code cuando country_name es igual en ambas filas
df_results = sustituir_valor_filtrado(df_results, 'country_code', 'country_name')

# Si aun existen datos en la columna country_code vacíos, les asignamos el valor de la columna country_3_letter_code
df_results['country_code'] = df_results['country_code'].replace('', pd.NA)  # Convertir cadenas vacías a NaN
df_results['country_code'] = df_results['country_code'].fillna(df_results['country_3_letter_code'])

In [11]:
# Rellenamos todas las url nulas o vacías con la url que tiene el atleta en el dataset de atletas
df_results = llenar_url_faltante(df_results, df_atletas)

# Remplazamos columna value_type nulas por columna value_type no nulas cuando discipline_title es igual en ambas filas
df_results = sustituir_valor_filtrado(df_results, 'value_type', 'discipline_title')

# Si aún existen datos en la columna value_type nulos les asignamos el valor NT (No Type)
df_results['value_type'] = df_results['value_type'].fillna('NT')

# Sustituimos value_unit por NV (No Value) cuando es nulo
df_results['value_unit'] = df_results['value_unit'].fillna('NV')

# Por último rellenamos los valores nulos faltantes con el dato vacío ''
df_results = df_results.fillna('')

verificar_limpieza(df_results)

Datos Nulos:
discipline_title         0
event_title              0
slug_game                0
participant_type         0
medal_type               0
athletes                 0
rank_equal               0
rank_position            0
country_name             0
country_code             0
country_3_letter_code    0
athlete_url              0
athlete_full_name        0
value_unit               0
value_type               0
dtype: int64


Tipos de Datos:
discipline_title         object
event_title              object
slug_game                object
participant_type         object
medal_type               object
athletes                 object
rank_equal                 bool
rank_position            object
country_name             object
country_code             object
country_3_letter_code    object
athlete_url              object
athlete_full_name        object
value_unit               object
value_type               object
dtype: object


Limpieza Dataset Medallas

In [12]:
analizar_dataframe(df_medals, "Medallas")

====== Dataset: Medallas ======
Primeras 10 filas:


,discipline_title,slug_game,event_title,event_gender,medal_type,participant_type,participant_title,athlete_url,athlete_full_name,country_name,country_code,country_3_letter_code
0,Curling,beijing-2022,Mixed Doubles,Mixed,GOLD,GameTeam,Italy,https://olympics.com/en/athletes/stefania-cons...,Stefania CONSTANTINI,Italy,IT,ITA
1,Curling,beijing-2022,Mixed Doubles,Mixed,GOLD,GameTeam,Italy,https://olympics.com/en/athletes/amos-mosaner,Amos MOSANER,Italy,IT,ITA
2,Curling,beijing-2022,Mixed Doubles,Mixed,SILVER,GameTeam,Norway,https://olympics.com/en/athletes/kristin-skaslien,Kristin SKASLIEN,Norway,NO,NOR
3,Curling,beijing-2022,Mixed Doubles,Mixed,SILVER,GameTeam,Norway,https://olympics.com/en/athletes/magnus-nedreg...,Magnus NEDREGOTTEN,Norway,NO,NOR
4,Curling,beijing-2022,Mixed Doubles,Mixed,BRONZE,GameTeam,Sweden,https://olympics.com/en/athletes/almida-de-val,Almida DE VAL,Sweden,SE,SWE
5,Curling,beijing-2022,Mixed Doubles,Mixed,BRONZE,GameTeam,Sweden,https://olympics.com/en/athletes/oskar-eriksson,Oskar ERIKSSON,Sweden,SE,SWE
6,Curling,beijing-2022,Women,Women,GOLD,GameTeam,Great Britain,NaN,NaN,Great Britain,GB,GBR
7,Curling,beijing-2022,Women,Women,SILVER,GameTeam,Japan,NaN,NaN,Japan,JP,JPN
8,Curling,beijing-2022,Women,Women,BRONZE,GameTeam,Sweden,NaN,NaN,Sweden,SE,SWE
9,Curling,beijing-2022,Men,Men,GOLD,GameTeam,Sweden,NaN,NaN,Sweden,SE,SWE



Información Básica del DataFrame: Medallas
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21697 entries, 0 to 21696
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   discipline_title       21697 non-null  object
 1   slug_game              21697 non-null  object
 2   event_title            21697 non-null  object
 3   event_gender           21697 non-null  object
 4   medal_type             21697 non-null  object
 5   participant_type       21697 non-null  object
 6   participant_title      6584 non-null   object
 7   athlete_url            17027 non-null  object
 8   athlete_full_name      18073 non-null  object
 9   country_name           21697 non-null  object
 10  country_code           20195 non-null  object
 11  country_3_letter_code  21697 non-null  object
dtypes: object(12)
memory usage: 2.0+ MB

Descripción Estadística:


,discipline_title,slug_game,event_title,event_gender,medal_type,participant_type,participant_title,athlete_url,athlete_full_name,country_name,country_code,country_3_letter_code
count,21697,21697,21697,21697,21697,21697,6584,17027,18073,21697,20195,21697
unique,86,53,1436,4,3,2,493,12116,12895,154,143,154
top,Athletics,tokyo-2020,Individual men,Men,BRONZE,Athlete,United States team,https://olympics.com/en/athletes/michael-phelp...,Michael PHELPS,United States of America,US,USA
freq,3080,1188,215,13932,7529,15113,523,16,16,3094,3094,3094



Datos de Columnas:


,Nombre,Tipo,Cantidad de Valores,Valores Únicos,Nulos,Strings Vacíos,Más Frecuente,Menos Frecuente
0,discipline_title,object,21697,86,0,0,Athletics,Cricket
1,slug_game,object,21697,53,0,0,tokyo-2020,st-moritz-1928
2,event_title,object,21697,1436,0,0,Individual men,class B up to 60 feet men
3,event_gender,object,21697,4,0,0,Men,Mixed
4,medal_type,object,21697,3,0,0,BRONZE,SILVER
5,participant_type,object,21697,2,0,0,Athlete,GameTeam
6,participant_title,object,21697,493,15113,0,United States team,Crabe II #4
7,athlete_url,object,21697,12116,4670,0,https://olympics.com/en/athletes/michael-phelp...,https://olympics.com/en/athletes/stefania-cons...
8,athlete_full_name,object,21697,12895,3624,0,Michael PHELPS,Alexandros Nikolopoulos
9,country_name,object,21697,154,0,0,United States of America,Turkmenistan


In [13]:
# Limpiamos posibles espacios en blanco al principio y final de strings
df_medals = trim_strings(df_medals)

In [14]:
# Remplazamos valores vacíos en la columna country_code por valores country_code cuando country_3_letter_code es igual en ambas filas
df_medals = sustituir_valor_filtrado(df_medals, 'country_code', 'country_3_letter_code')

# Remplazamos valores vacíos restantes en la columna country_code por valores country_code cuando country_name es igual en ambas filas
df_medals = sustituir_valor_filtrado(df_medals, 'country_code', 'country_name')

# Si aun existen datos en la columna country_code vacíos, les asignamos el valor de la columna country_3_letter_code
df_medals['country_code'] = df_medals['country_code'].replace('', pd.NA)  # Convertir cadenas vacías a NaN
df_medals['country_code'] = df_medals['country_code'].fillna(df_medals['country_3_letter_code'])

In [15]:
generar_df_resumen(df_medals)

,Nombre,Tipo,Cantidad de Valores,Valores Únicos,Nulos,Strings Vacíos,Más Frecuente,Menos Frecuente
0,discipline_title,object,21697,86,0,0,Athletics,Cricket
1,slug_game,object,21697,53,0,0,tokyo-2020,st-moritz-1928
2,event_title,object,21697,1436,0,0,Individual men,class B up to 60 feet men
3,event_gender,object,21697,4,0,0,Men,Mixed
4,medal_type,object,21697,3,0,0,BRONZE,SILVER
5,participant_type,object,21697,2,0,0,Athlete,GameTeam
6,participant_title,object,21697,493,15113,0,United States team,Crabe II #4
7,athlete_url,object,21697,12116,4670,0,https://olympics.com/en/athletes/michael-phelp...,https://olympics.com/en/athletes/stefania-cons...
8,athlete_full_name,object,21697,12895,3624,0,Michael PHELPS,Alexandros Nikolopoulos
9,country_name,object,21697,154,0,0,United States of America,Turkmenistan


In [16]:
#Rellenamos las url nulas o vacías con la url que tiene el atleta en el dataset de atletas
df_medals = llenar_url_faltante(df_medals, df_atletas)

#Rellenamos los valores nulos faltantes con el dato vacío ''
df_medals = df_medals.fillna('')

verificar_limpieza(df_medals)

Datos Nulos:
discipline_title         0
slug_game                0
event_title              0
event_gender             0
medal_type               0
participant_type         0
participant_title        0
athlete_url              0
athlete_full_name        0
country_name             0
country_code             0
country_3_letter_code    0
dtype: int64


Tipos de Datos:
discipline_title         object
slug_game                object
event_title              object
event_gender             object
medal_type               object
participant_type         object
participant_title        object
athlete_url              object
athlete_full_name        object
country_name             object
country_code             object
country_3_letter_code    object
dtype: object


Limpieza Dataset Atletas

In [17]:
analizar_dataframe(df_atletas, "Atletas")

====== Dataset: Atletas ======
Primeras 10 filas:


,athlete_url,athlete_full_name,games_participations,first_game,athlete_year_birth,athlete_medals,bio
0,https://olympics.com/en/athletes/cooper-woods-...,Cooper WOODS-TOPALOVIC,1,Beijing 2022,2000.0,NaN,NaN
1,https://olympics.com/en/athletes/elofsson,Felix ELOFSSON,2,PyeongChang 2018,1995.0,NaN,NaN
2,https://olympics.com/en/athletes/dylan-walczyk,Dylan WALCZYK,1,Beijing 2022,1993.0,NaN,NaN
3,https://olympics.com/en/athletes/olli-penttala,Olli PENTTALA,1,Beijing 2022,1995.0,NaN,NaN
4,https://olympics.com/en/athletes/reikherd,Dmitriy REIKHERD,1,Beijing 2022,1989.0,NaN,NaN
5,https://olympics.com/en/athletes/matt-graham,Matt GRAHAM,3,Sochi 2014,1994.0,\n\n\n1\n\nS\n\n,NaN
6,https://olympics.com/en/athletes/ikuma-horishima,Ikuma HORISHIMA,2,PyeongChang 2018,1997.0,\n\n\n1\n\nB\n\n,NaN
7,https://olympics.com/en/athletes/daichi-hara,Daichi HARA,2,PyeongChang 2018,1997.0,\n\n\n1\n\nB\n\n,NaN
8,https://olympics.com/en/athletes/laurent-dumais,Laurent DUMAIS,1,Beijing 2022,1996.0,NaN,NaN
9,https://olympics.com/en/athletes/james-matheson,James MATHESON,2,PyeongChang 2018,1995.0,NaN,NaN



Información Básica del DataFrame: Atletas
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75904 entries, 0 to 75903
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   athlete_url           75904 non-null  object 
 1   athlete_full_name     75904 non-null  object 
 2   games_participations  75904 non-null  int64  
 3   first_game            75882 non-null  object 
 4   athlete_year_birth    73448 non-null  float64
 5   athlete_medals        15352 non-null  object 
 6   bio                   22842 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 4.1+ MB

Descripción Estadística:


,athlete_url,athlete_full_name,games_participations,first_game,athlete_year_birth,athlete_medals,bio
count,75904,75904,75904.000000,75882,73448.000000,15352,22842
unique,75900,75480,NaN,53,NaN,170,22530
top,https://olympics.com/en/athletes/martin-bidar,Hao WANG,NaN,Rio 2016,NaN,\n\n\n1\n\nB\n\n,\n\n\nPersonal Best: Mar – unknown.\n\n\n\n\n\n
freq,2,4,NaN,4111,NaN,4209,110
mean,NaN,NaN,1.535874,NaN,1961.619377,NaN,NaN
std,NaN,NaN,0.854563,NaN,28.129576,NaN,NaN
min,NaN,NaN,0.000000,NaN,1836.000000,NaN,NaN
25%,NaN,NaN,1.000000,NaN,1946.000000,NaN,NaN
50%,NaN,NaN,1.000000,NaN,1968.000000,NaN,NaN
75%,NaN,NaN,2.000000,NaN,1983.000000,NaN,NaN



Datos de Columnas:


,Nombre,Tipo,Cantidad de Valores,Valores Únicos,Nulos,Strings Vacíos,Más Frecuente,Menos Frecuente
0,athlete_url,object,75904,75900,0,0,https://olympics.com/en/athletes/belorukova,https://olympics.com/en/athletes/laurent-dumais
1,athlete_full_name,object,75904,75480,0,0,Francisco SANCHEZ,Tessa MAUD
2,games_participations,int64,75904,11,0,0,1,10
3,first_game,object,75904,53,22,0,Rio 2016,Chamonix 1924
4,athlete_year_birth,float64,75904,165,2456,0,1985.0,2009.0
5,athlete_medals,object,75904,170,60552,0,\n\n\n1\n\nB\n\n,\n\n\n2\n\nG\n\n\n\n4\n\nS\n\n\n\n5\n\nB\n\n
6,bio,object,75904,22530,53062,0,\n\n\nPersonal Best: Mar – unknown.\n\n\n\n\n\n,\n\n\nA bronze medallist for her country at th...


In [18]:
# Convertimos los años a un tipo de dato más específico, manteniendo los nulos como NaN
df_atletas['athlete_year_birth'] = df_atletas['athlete_year_birth'].astype('Int64')

# Limpiamos posibles espacios en blanco al principio y final de strings
df_atletas = trim_strings(df_atletas)

verificar_limpieza(df_atletas)

Datos Nulos:
athlete_url                 0
athlete_full_name           0
games_participations        0
first_game                 22
athlete_year_birth       2456
athlete_medals          60552
bio                     53062
dtype: int64


Tipos de Datos:
athlete_url             object
athlete_full_name       object
games_participations     int64
first_game              object
athlete_year_birth       Int64
athlete_medals          object
bio                     object
dtype: object


In [19]:
# Corregimos el nombre "slug_game" a "game_slug" para coincidir con el DataFrame de hosts
df_results.rename(columns={'slug_game': 'game_slug'}, inplace=True)

In [20]:
# Identificamos atletas con el dato de first_game nulos. 
# Nos proponemos usar el DataFrame de Results para completar este dato
# usando su primera participación según los resultados individuales y grupales

# Filtrar filas con valores nulos en 'first_game'
df_atletas_sin_first_game = df_atletas[pd.isnull(df_atletas['first_game'])]

# Iterar sobre cada fila con 'first_game' nulo
for i in range(len(df_atletas_sin_first_game)):
    fila_atleta = df_atletas_sin_first_game.iloc[i]
    
    # Filtrar df_results por participaciones individuales y grupales del atleta
    df_juegos_atleta = df_results[
        (df_results['athlete_full_name'] == fila_atleta['athlete_full_name']) | # Nombre en athlete_full_name
        (df_results['athletes'].apply(lambda x: isinstance(x, list) and any(athlete[0] == fila_atleta['athlete_full_name'] for athlete in x))) # Nombre como parte de lista de atletas en athletes
    ]
    # Si se encontraron 1 o más resultados para el atleta, se asigna el game_slug con el año más bajo como first_game 
    if not df_juegos_atleta.empty: 
        # Añadir columna 'year' desde df_hosts
        df_juegos_atleta = df_juegos_atleta.merge(df_hosts, on='game_slug', how='left')
        df_juego_mas_antiguo = df_juegos_atleta.loc[df_juegos_atleta['game_year'].idxmin()]
        primer_juego = df_juego_mas_antiguo['game_slug']
        
        # Actualizar 'first_game' en df_atletas usando loc
        df_atletas.loc[df_atletas['athlete_full_name'] == fila_atleta['athlete_full_name'], 'first_game'] = primer_juego


In [21]:
# Descartamos dato "bio", que no es de valor en este proyecto
df_atletas.drop(columns=['bio'],inplace=True)

In [22]:
# Para atletas sin fecha de nacimiento, se crea fecha de nacimiento estimada 
# con promedio de años nacimientos para atletas que comparten el mismo primer juego

# Agregar columna para indicar si el año de nacimiento es estimado
df_atletas['birth_year_estimated'] = False

# Filtrar atletas sin año de nacimiento
df_atletas_sin_nacimiento = df_atletas[pd.isnull(df_atletas['athlete_year_birth'])]

# Filtrar atletas con año de nacimiento
df_atletas_con_nacimiento = df_atletas[pd.notna(df_atletas['athlete_year_birth'])]

# Iterar sobre atletas sin año de nacimiento
for i in range(len(df_atletas_sin_nacimiento)):
    atleta_sin_nacimiento = df_atletas_sin_nacimiento.iloc[i]
    
    # Filtrar atletas con el mismo primer juego
    df_atletas_mismo_primer_juego = df_atletas_con_nacimiento[df_atletas_con_nacimiento['first_game'] == atleta_sin_nacimiento['first_game']]
    
    if not df_atletas_mismo_primer_juego.empty: 
        # Calcular el promedio del año de nacimiento y convertirlo a entero
        promedio_nacimiento = int(df_atletas_mismo_primer_juego['athlete_year_birth'].mean())
        
        # Actualizar la columna 'athlete_year_birth' con el promedio
        df_atletas.loc[df_atletas['athlete_full_name'] == atleta_sin_nacimiento['athlete_full_name'], 'athlete_year_birth'] = promedio_nacimiento
        # Marcar el año de nacimiento como estimado
        df_atletas.loc[df_atletas['athlete_full_name'] == atleta_sin_nacimiento['athlete_full_name'], 'birth_year_estimated'] = True


verificar_limpieza(df_atletas)

Datos Nulos:
athlete_url                 0
athlete_full_name           0
games_participations        0
first_game                  0
athlete_year_birth          3
athlete_medals          60552
birth_year_estimated        0
dtype: int64


Tipos de Datos:
athlete_url             object
athlete_full_name       object
games_participations     int64
first_game              object
athlete_year_birth       Int64
athlete_medals          object
birth_year_estimated      bool
dtype: object


In [23]:
df_atletas

,athlete_url,athlete_full_name,games_participations,first_game,athlete_year_birth,athlete_medals,birth_year_estimated
0,https://olympics.com/en/athletes/cooper-woods-...,Cooper WOODS-TOPALOVIC,1,Beijing 2022,2000,NaN,False
1,https://olympics.com/en/athletes/elofsson,Felix ELOFSSON,2,PyeongChang 2018,1995,NaN,False
2,https://olympics.com/en/athletes/dylan-walczyk,Dylan WALCZYK,1,Beijing 2022,1993,NaN,False
3,https://olympics.com/en/athletes/olli-penttala,Olli PENTTALA,1,Beijing 2022,1995,NaN,False
4,https://olympics.com/en/athletes/reikherd,Dmitriy REIKHERD,1,Beijing 2022,1989,NaN,False
...,...,...,...,...,...,...,...
75899,https://olympics.com/en/athletes/douglas-weigle,Douglas WEIGLE,1,Innsbruck 1976,1955,NaN,False
75900,https://olympics.com/en/athletes/stefania-bertele,Stefania BERTELE,1,Innsbruck 1976,1957,NaN,False
75901,https://olympics.com/en/athletes/walter-cecconi,Walter CECCONI,1,Innsbruck 1976,1957,NaN,False
75902,https://olympics.com/en/athletes/susan-kelley,Susan KELLEY,1,Innsbruck 1976,1954,NaN,False


In [24]:
# Decidimos usar los datos de medallas del DataFrame Medals, en vez del dato de medallas en el DataFrame de atletas
# el dato en DataFrame Medals se presenta en un formato más claro y es más completo.

# La columna athlete_medals se usará para contener un entero con la cantidad total de medallas.
# También se agregarán columnas para las cantidades de cada tipo de medalla.

df_atletas.drop(columns=['athlete_medals'],inplace=True)
# Agrupar por athlete_full_name y contar las medallas de cada tipo
medals_summary = df_medals.groupby('athlete_full_name').agg(
    athlete_medals=pd.NamedAgg(column='medal_type', aggfunc='count'),
    gold_medals=pd.NamedAgg(column='medal_type', aggfunc=lambda x: (x == 'GOLD').sum()),
    silver_medals=pd.NamedAgg(column='medal_type', aggfunc=lambda x: (x == 'SILVER').sum()),
    bronze_medals=pd.NamedAgg(column='medal_type', aggfunc=lambda x: (x == 'BRONZE').sum())
).reset_index()

# Unir el resumen de medallas con el DataFrame de atletas
df_atletas = df_atletas.merge(medals_summary, on='athlete_full_name', how='left')

# Rellenar NaN con 0 para las columnas de medallas
df_atletas[['athlete_medals', 'gold_medals', 'silver_medals', 'bronze_medals']] = df_atletas[['athlete_medals', 'gold_medals', 'silver_medals', 'bronze_medals']].fillna(0).astype(int)


In [25]:
verificar_limpieza(df_atletas)

Datos Nulos:
athlete_url             0
athlete_full_name       0
games_participations    0
first_game              0
athlete_year_birth      3
birth_year_estimated    0
athlete_medals          0
gold_medals             0
silver_medals           0
bronze_medals           0
dtype: int64


Tipos de Datos:
athlete_url             object
athlete_full_name       object
games_participations     int64
first_game              object
athlete_year_birth       Int64
birth_year_estimated      bool
athlete_medals           int64
gold_medals              int64
silver_medals            int64
bronze_medals            int64
dtype: object


In [26]:
analizar_dataframe(df_atletas, "Atletas")

====== Dataset: Atletas ======
Primeras 10 filas:


,athlete_url,athlete_full_name,games_participations,first_game,athlete_year_birth,birth_year_estimated,athlete_medals,gold_medals,silver_medals,bronze_medals
0,https://olympics.com/en/athletes/cooper-woods-...,Cooper WOODS-TOPALOVIC,1,Beijing 2022,2000,False,0,0,0,0
1,https://olympics.com/en/athletes/elofsson,Felix ELOFSSON,2,PyeongChang 2018,1995,False,0,0,0,0
2,https://olympics.com/en/athletes/dylan-walczyk,Dylan WALCZYK,1,Beijing 2022,1993,False,0,0,0,0
3,https://olympics.com/en/athletes/olli-penttala,Olli PENTTALA,1,Beijing 2022,1995,False,0,0,0,0
4,https://olympics.com/en/athletes/reikherd,Dmitriy REIKHERD,1,Beijing 2022,1989,False,0,0,0,0
5,https://olympics.com/en/athletes/matt-graham,Matt GRAHAM,3,Sochi 2014,1994,False,1,0,1,0
6,https://olympics.com/en/athletes/ikuma-horishima,Ikuma HORISHIMA,2,PyeongChang 2018,1997,False,1,0,0,1
7,https://olympics.com/en/athletes/daichi-hara,Daichi HARA,2,PyeongChang 2018,1997,False,1,0,0,1
8,https://olympics.com/en/athletes/laurent-dumais,Laurent DUMAIS,1,Beijing 2022,1996,False,0,0,0,0
9,https://olympics.com/en/athletes/james-matheson,James MATHESON,2,PyeongChang 2018,1995,False,0,0,0,0



Información Básica del DataFrame: Atletas
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75904 entries, 0 to 75903
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   athlete_url           75904 non-null  object
 1   athlete_full_name     75904 non-null  object
 2   games_participations  75904 non-null  int64 
 3   first_game            75904 non-null  object
 4   athlete_year_birth    75901 non-null  Int64 
 5   birth_year_estimated  75904 non-null  bool  
 6   athlete_medals        75904 non-null  int64 
 7   gold_medals           75904 non-null  int64 
 8   silver_medals         75904 non-null  int64 
 9   bronze_medals         75904 non-null  int64 
dtypes: Int64(1), bool(1), int64(5), object(3)
memory usage: 5.4+ MB

Descripción Estadística:


,athlete_url,athlete_full_name,games_participations,first_game,athlete_year_birth,birth_year_estimated,athlete_medals,gold_medals,silver_medals,bronze_medals
count,75904,75904,75904.000000,75904,75901.0,75904,75904.000000,75904.000000,75904.000000,75904.000000
unique,75900,75480,NaN,68,<NA>,2,NaN,NaN,NaN,NaN
top,https://olympics.com/en/athletes/martin-bidar,Hao WANG,NaN,Rio 2016,<NA>,False,NaN,NaN,NaN,NaN
freq,2,4,NaN,4110,<NA>,73443,NaN,NaN,NaN,NaN
mean,NaN,NaN,1.535874,NaN,1960.099248,NaN,0.226589,0.074173,0.073224,0.079192
std,NaN,NaN,0.854563,NaN,29.591071,NaN,0.631879,0.341431,0.298542,0.301745
min,NaN,NaN,0.000000,NaN,1836.0,NaN,0.000000,0.000000,0.000000,0.000000
25%,NaN,NaN,1.000000,NaN,1944.0,NaN,0.000000,0.000000,0.000000,0.000000
50%,NaN,NaN,1.000000,NaN,1967.0,NaN,0.000000,0.000000,0.000000,0.000000
75%,NaN,NaN,2.000000,NaN,1983.0,NaN,0.000000,0.000000,0.000000,0.000000



Datos de Columnas:


,Nombre,Tipo,Cantidad de Valores,Valores Únicos,Nulos,Strings Vacíos,Más Frecuente,Menos Frecuente
0,athlete_url,object,75904,75900,0,0,https://olympics.com/en/athletes/belorukova,https://olympics.com/en/athletes/laurent-dumais
1,athlete_full_name,object,75904,75480,0,0,Francisco SANCHEZ,Tessa MAUD
2,games_participations,int64,75904,11,0,0,1,10
3,first_game,object,75904,68,0,0,Rio 2016,turin-2006
4,athlete_year_birth,Int64,75904,165,3,0,1985,2009
5,birth_year_estimated,bool,75904,2,0,0,False,True
6,athlete_medals,int64,75904,14,0,0,0,16
7,gold_medals,int64,75904,10,0,0,0,13
8,silver_medals,int64,75904,6,0,0,0,5
9,bronze_medals,int64,75904,6,0,0,0,5


Guardamos los DataFrames limpios y mejorados en el área refined

In [27]:
df_hosts.to_csv('datos/refined/olympic_hosts.csv', index=False)
df_atletas.to_csv('datos/refined/olympic_athletes.csv', index=False)
df_medals.to_csv('datos/refined/olympic_medals.csv', index=False)
df_results.to_csv('datos/refined/olympic_results.csv', index=False)